http://localhost:8888/notebooks/PJ/Multicampus/Crawl/%ED%81%B4%EB%9E%98%EC%8A%A4%EB%A5%BC%20%EB%A7%8C%EB%93%A4%EC%96%B4%20%EB%B3%B4%EC%9E%90.ipynb

# 질문
> 순서도 따라서 어떻게 해야 되는지

1. 날짜 뱉어내
2. 날짜를 넘겨
3. 트윗 검색 조건 설정해

---

이건 내가 가능하려나 싶은

- getJsondata 안에서 바로 바로 읽어올 때마다 하는 방식은?
- 체크포인트,,,

----- 대충 해결------
- HTTPError 429 : fakeuseragent 해결 가능한지 확인
    - 일단 time.sleep(3) 정도면 트위터가 봐주는듯
    - 애초에 여기 헤더에 있는 헤더가 랜덤으로 헤더 가져오고, 거의 fake useragent에 있는 애들이다. 굳이 건들지 않아도 될듯?

-------- 해결 ㄴㄴ-----------

- 제너레이터 만들어서 날짜 뱉어내고, 하루 단위로 실행하도록 만들기.
- 날짜 너무 비효율적인디?
- 제발 클래스로...
- retry 데코레이터 ㅇㅇㅇ
- getJsondata 안에서 바로 바로 읽어올 때마다 하는 방식은?
- 체크포인트,,,

- 함수 바꾼 부분 정리해/놓기!!!!!
- 스태틱 메소드니까 상관 ㄴㄴㄴㄴㄴㄴ
- logger 찍어야 하는디


-------- adsp 끝나고 ----------


- retry while문 구현 실패. 아마 패키지 자체가 tweet 가져오는 과정에서 뭔가 있는듯한데 어떻게 소스 확인해야할지 고민


- 패키지 코드 다시 확인 : Tweet 객체 안에서 tqdm 찍게 못 바꾸나.
    - 일단 지금까지 확인한 건 tweet 객체 생성을 위해 트위터 스크롤 다 내리는 방식인듯한디.
    - 애초에 내가 확인하고 싶은 건 크롤링 진행 상황. 이게 오래 걸리는 건데 밑에서 tqdm 찍는 건 개 빨라서 굳이 의미 ㄴㄴ.

In [1]:
# install GOT3 package
!pip install GetOldTweets3

In [2]:
# module import

import GetOldTweets3 as got
from bs4 import BeautifulSoup

import sys
import urllib
import json
from time import sleep

import datetime
import time
import os
from random import uniform
from tqdm import tqdm
import csv

In [3]:
def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
    """
    Invoke an HTTP query to Twitter.
    Should not be used as an API function. A static method.
    """
    url = "https://twitter.com/i/search/timeline?"

    if not tweetCriteria.topTweets:
        url += "f=tweets&"

    url += ("vertical=news&q=%s&src=typd&%s"
            "&include_available_features=1&include_entities=1&max_position=%s"
            "&reset_error_state=false")

    urlGetData = ''

    if hasattr(tweetCriteria, 'querySearch'):
        urlGetData += tweetCriteria.querySearch

    if hasattr(tweetCriteria, 'excludeWords'):
        urlGetData += ' -'.join([''] + tweetCriteria.excludeWords)

    if hasattr(tweetCriteria, 'username'):
        if not hasattr(tweetCriteria.username, '__iter__'):
            tweetCriteria.username = [tweetCriteria.username]

        usernames_ = [u.lstrip('@') for u in tweetCriteria.username if u]
        tweetCriteria.username = {u.lower() for u in usernames_ if u}

        usernames = [' from:'+u for u in sorted(tweetCriteria.username)]
        if usernames:
            urlGetData += ' OR'.join(usernames)

    if hasattr(tweetCriteria, 'within'):
        if hasattr(tweetCriteria, 'near'):
            urlGetData += ' near:"%s" within:%s' % (tweetCriteria.near, tweetCriteria.within)
        elif hasattr(tweetCriteria, 'lat') and hasattr(tweetCriteria, 'lon'):
            urlGetData += ' geocode:%f,%f,%s' % (tweetCriteria.lat, tweetCriteria.lon, tweetCriteria.within)

    if hasattr(tweetCriteria, 'since'):
        urlGetData += ' since:' + tweetCriteria.since

    if hasattr(tweetCriteria, 'until'):
        urlGetData += ' until:' + tweetCriteria.until

    if hasattr(tweetCriteria, 'minReplies'):
        urlGetData += ' min_replies:' + tweetCriteria.minReplies

    if hasattr(tweetCriteria, 'minFaves'):
        urlGetData += ' min_faves:' + tweetCriteria.minFaves

    if hasattr(tweetCriteria, 'minRetweets'):
        urlGetData += ' min_retweets:' + tweetCriteria.minRetweets

    if hasattr(tweetCriteria, 'lang'):
        urlLang = 'l=' + tweetCriteria.lang + '&'
    else:
        urlLang = ''
    url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
    useragent = useragent or TweetManager.user_agents[0]

    headers = [
        ('Host', "twitter.com"),
        ('User-Agent', useragent),
        ('Accept', "application/json, text/javascript, */*; q=0.01"),
        ('Accept-Language', "en-US,en;q=0.5"),
        ('X-Requested-With', "XMLHttpRequest"),
        ('Referer', url),
        ('Connection', "keep-alive")
    ]

    if proxy:
        opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
    else:
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
    opener.addheaders = headers

    if debug:
        print(url)
        # print('\n'.join(h[0]+': '+h[1] for h in headers))

    # HTTP Request 429 : 타임 슬립 줘보자
    time.sleep(3)

    try:
        response = opener.open(url)
        jsonResponse = response.read()

    except TimeoutError as e:
        print("Timeout error")
        print('sleep 30')
        time.sleep(30)

        #이 부분 나중에 retry로 진행하기
        try:
            response = opener.open(url)
            jsonResponse = response.read()
        except TimeoutError as e:
            print("Timeout Error again.")
            print("Pass Data!!!!!!!!!")
            pass

#     count = 0
#     while True:
#         try:
#             response = opener.open(url)
#             jsonResponse = response.read()
        
#             # 완료시 break 에러시 except 문 실행.
#             break
            
#         except TimeOutError as e:
#             print("Timeout error")
#             count+=1
#             if count >=3:
#                 break
#             time.sleep(30)

#     if count >=3:
#         continue

    except Exception as e:
        print("An error occured during an HTTP request:", str(e))
        print("Error URL:", url)
        print("Try to open in browser: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))

        # 이 부분 나중에 확인하기
        print("sleep 30!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        pass
        # sys.exit()
        

    try:
        s_json = jsonResponse.decode()
    except:
        print("Invalid response from Twitter")
        print("Error URL:", url)
        pass
        # sys.exit()
    else:
        try:
            dataJson = json.loads(s_json)
        except:
            print("Error parsing JSON: %s" % s_json)
            print("Error URL:", url)
        pass
        # sys.exit()

#     if debug:
#         print(s_json)
#         print("---\n")

    return dataJson

In [4]:
# 날짜 다시 입력해야 할 때 에러 발생시키자.
class NotValidEndDateError(Exception):
    def __init__(self):
        super().__init__('마지막 검색 날짜를 다시 설정하십시오.')

In [5]:
# GetOldTweets3 패키지의 setUntil 메소드는 마지막 날짜 포함하지 않으므로, 주의.
def set_crawl_date(start_date, end_date):
    
    start_date = datetime.datetime.strptime(str(start_date), "%Y%m%d")
    end_date = datetime.datetime.strptime(str(end_date), "%Y%m%d") - datetime.timedelta(days=1)
    
    if end_date == start_date:
        raise NotValidEndDateError
    
    else:   
        print("트윗 수집 날짜 설정: {0}부터 {1}까지".format(start_date, end_date))    
        return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

In [6]:
def crawl_tweets(start_date, end_date, debug=False):    
    got.manager.TweetManager.getJsonResponse = getJsonResponse # 함수에 이 부분 넣어주고
    
    print("========== 트윗 수집 시작: {0} ~ {1} ==========".format(start_date, end_date))
    start_time = time.time()
    tweet_criteria = got.manager.TweetCriteria().setQuerySearch('Elon Musk')\
                                                .setSince(start_date)\
                                                .setUntil(end_date)\
                                                .setLang('en')
    tweets = got.manager.TweetManager.getTweets(tweet_criteria, debug=debug)
    
    elapsed_time = time.time()-start_time
    
    print("수집 완료 : {}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    print("총 수집 트윗 개수 : {0}".format(len(tweets)))
    
    return tweets

In [7]:
def get_results(tweet_data):
    results = []
    for tweet in tqdm(tweet_data):
        results.append({'url': tweet.permalink,
                        'date': tweet.date,
                        'text': tweet.text,
                        'user': tweet.username,
                        'mentions': tweet.mentions,
                        'retweets': tweet.retweets,
                        'favorites': tweet.favorites,
                        'hashtags': tweet.hashtags})
        
    return results        

In [8]:
def save_tweets(tweet_lists):
    base_file_dir = "tweets"

    if not os.path.exists(base_file_dir):
        os.makedirs(base_file_dir)
        
    with open(f"{base_file_dir}/tweets_{crawl_start}_{crawl_end}.csv", "a", -1, encoding="utf-8") as f:    
        writer = csv.writer(f)
        writer.writerow(['url', 'date', 'text', 'user', 'mentions', 'retweets', 'favorites', 'hashtags'])        
        for tweet_list in tqdm(tweet_lists):
            writer.writerow(list(tweet_list.values()))
            
    return 

In [28]:
crawl_start, crawl_end = set_crawl_date(20200201, 20200203)

트윗 수집 날짜 설정: 2020-02-01 00:00:00부터 2020-02-02 00:00:00까지


In [29]:
tweet_results = crawl_tweets(crawl_start, crawl_end)

========== 트윗 수집 시작: 2020-02-01 ~ 2020-02-02 ==========
수집 완료 : 01:00:26
총 수집 트윗 개수 : 11232


In [30]:
tweet_results_lists = get_results(tweet_results)

100%|████████████████████████████████████████████████████████████████████████| 11232/11232 [00:00<00:00, 196353.15it/s]


In [31]:
save_tweets(tweet_results_lists)

100%|█████████████████████████████████████████████████████████████████████████| 11232/11232 [00:00<00:00, 49620.79it/s]


In [34]:
crawl_start, crawl_end = set_crawl_date(20200601, 20200603)

트윗 수집 날짜 설정: 2020-06-01 00:00:00부터 2020-06-02 00:00:00까지


In [35]:
tweet_results = crawl_tweets(crawl_start, crawl_end)

========== 트윗 수집 시작: 2020-06-01 ~ 2020-06-02 ==========
수집 완료 : 01:31:45
총 수집 트윗 개수 : 22646


In [36]:
tweet_results_lists = get_results(tweet_results)

100%|████████████████████████████████████████████████████████████████████████| 22646/22646 [00:00<00:00, 509678.57it/s]


In [37]:
save_tweets(tweet_results_lists)

100%|████████████████████████████████████████████████████████████████████████| 22646/22646 [00:00<00:00, 105594.68it/s]


# retry 질문..

도대체 어떻게 쓰는거니 ㅠㅅㅠ 머리야 생각을해...

In [ ]:
# def retry(total_try_cnt=5, sleep_in_sec=5, retryable_exceptions=(TimeoutError)):
#     def decorator(getJsonResponse):
#         @functools.wraps(getJsonResponse)
#         def wrapper(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
#             for cnt in range(total_try_cnt):
#                 logger.info(f"trying {func.__name__}() [{cnt+1}/{total_try_cnt}]")

#                 try:
#                     result = func(*args, **kwargs)
#                     logger.info(f"in retry(), {func.__name__}() returned '{result}'")

#                     if result: 
#                         return result
#                 except retryable_exceptions as e:
#                     logger.info(f"in retry(), {func.__name__}() raised retryable exception '{e}'")
#                     pass
#                 except Exception as e:
#                     logger.info(f"in retry(), {func.__name__}() raised {e}")
#                     print(str(e))
#                     pass 
#                     # raise e
                    

#                 time.sleep(sleep_in_sec)
#             logger.info(f"{func.__name__} finally has been failed")
#         return wrapper
#     return decorator

NameError: name 'tweetCriteria' is not defined

--> 안에 넣어줘도 안 된다고 나온다...


`crawl_tweets(crawl_start, crawl_end)` 진행 시 오류

TypeError                                 Traceback (most recent call last)
<ipython-input-29-e4d276fc0ef7> in <module>()
----> 1 tweet_results = crawl_tweets(crawl_start, crawl_end)

TypeError: getJsonResponse() missing 2 required positional arguments: 'cookieJar' and 'proxy'


In [ ]:
# @retry(total_try_cnt=3, sleep_in_sec=3, retryable_exceptions=(TimeoutError))
# def my_method(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
#     rand = random.randint(0, 9)
#     logger.info(f"rand={rand}")

#     if rand % 3 == 0:
#         return True
#     elif rand % 3 == 1:
#         return False
#     else:
#         raise int("a")

# my_method(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False)

In [ ]:
# def retry(total_try_cnt=3, error_sleep=30, request_delay=2, retryable_exceptions=()):
    
#     def decorator(getJsonResponse):
#         @functools.wraps(getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False))
#         def getJsonResponse(tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False):
#             """
#             Invoke an HTTP query to Twitter.
#             Should not be used as an API function. A static method.
#             """
            
#             url = "https://twitter.com/i/search/timeline?"

#             # URL

#             if not tweetCriteria.topTweets:
#                 url += "f=tweets&"

#             url += ("vertical=news&q=%s&src=typd&%s"
#                     "&include_available_features=1&include_entities=1&max_position=%s"
#                     "&reset_error_state=false")

#             urlGetData = ''

#             if hasattr(tweetCriteria, 'querySearch'):
#                 urlGetData += tweetCriteria.querySearch

#             if hasattr(tweetCriteria, 'excludeWords'):
#                 urlGetData += ' -'.join([''] + tweetCriteria.excludeWords)

#             if hasattr(tweetCriteria, 'username'):
#                 if not hasattr(tweetCriteria.username, '__iter__'):
#                     tweetCriteria.username = [tweetCriteria.username]

#                 usernames_ = [u.lstrip('@') for u in tweetCriteria.username if u]
#                 tweetCriteria.username = {u.lower() for u in usernames_ if u}

#                 usernames = [' from:'+u for u in sorted(tweetCriteria.username)]
#                 if usernames:
#                     urlGetData += ' OR'.join(usernames)

#             if hasattr(tweetCriteria, 'within'):
#                 if hasattr(tweetCriteria, 'near'):
#                     urlGetData += ' near:"%s" within:%s' % (tweetCriteria.near, tweetCriteria.within)
#                 elif hasattr(tweetCriteria, 'lat') and hasattr(tweetCriteria, 'lon'):
#                     urlGetData += ' geocode:%f,%f,%s' % (tweetCriteria.lat, tweetCriteria.lon, tweetCriteria.within)

#             if hasattr(tweetCriteria, 'since'):
#                 urlGetData += ' since:' + tweetCriteria.since

#             if hasattr(tweetCriteria, 'until'):
#                 urlGetData += ' until:' + tweetCriteria.until

#             if hasattr(tweetCriteria, 'minReplies'):
#                 urlGetData += ' min_replies:' + tweetCriteria.minReplies

#             if hasattr(tweetCriteria, 'minFaves'):
#                 urlGetData += ' min_faves:' + tweetCriteria.minFaves

#             if hasattr(tweetCriteria, 'minRetweets'):
#                 urlGetData += ' min_retweets:' + tweetCriteria.minRetweets

#             if hasattr(tweetCriteria, 'lang'):
#                 urlLang = 'l=' + tweetCriteria.lang + '&'
#             else:
#                 urlLang = ''
            
#             url = url % (urllib.parse.quote(urlGetData.strip()), urlLang, urllib.parse.quote(refreshCursor))
#             useragent = useragent or TweetManager.user_agents[0]

#             headers = [
#                 ('Host', "twitter.com"),
#                 ('User-Agent', useragent),
#                 ('Accept', "application/json, text/javascript, */*; q=0.01"),
#                 ('Accept-Language', "en-US,en;q=0.5"),
#                 ('X-Requested-With', "XMLHttpRequest"),
#                 ('Referer', url),
#                 ('Connection', "keep-alive")
#             ]

#             if proxy:
#                 opener = urllib.request.build_opener(urllib.request.ProxyHandler({'http': proxy, 'https': proxy}), urllib.request.HTTPCookieProcessor(cookieJar))
#             else:
#                 opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookieJar))
#             opener.addheaders = headers

#             if debug:
#                 print(url)
#                 # print('\n'.join(h[0]+': '+h[1] for h in headers))

            
#             # HTTP Request 429 : 타임 슬립 줘보자
#             time.sleep(request_delay)

#             # JSON으로 정보 받아 오기

#             for cnt in range(total_try_cnt):

#                 logger.info(f"{getJsonResponse.__name__}() : {cnt+1} / {total_try_cnt}")
                
#                 try:
#                     response = opener.open(url)
#                     jsonResponse = response.read()
#                     logger.info(f"{getJsonResponse.__name__}() : {url}에서 데이터를 추출했습니다.")

#                 except retryable_exceptions as re:
#                     print(f"{re}: sleep {error_sleep}")
#                     time.sleep(error_sleep)                    
#                     logger.info(f"다시 시도: {getJsonResponse.__name__}()에서 {re}가 발생했습니다.")
                
#                 try:
#                     response = opener.open(url)
#                     jsonResponse = response.read()
                

#                 # except TimeoutError as e:
#                 #     print("Timeout error")
#                 #     print('sleep 30')
#                 #     time.sleep(30)

#                 #     # 이 부분 나중에 retry로 진행하기
#                 #     try:
#                 #         response = opener.open(url)
#                 #         jsonResponse = response.read()
#                 #     except TimeoutError as e:
#                 #         print("Timeout Error again.")
#                 #         print("Pass Data")
#                 #         pass

#                 except Exception as e:
#                     print("An error occured during an HTTP request:", str(e))
#                     print("Try to open in browser: https://twitter.com/search?q=%s&src=typd" % urllib.parse.quote(urlGetData))
#                     logger.info(f"{getJsonResponse.__name__}(): {e} 발생, url: {url}")
#                     # 이 부분 나중에 확인하기
#                     print("sleep {error_sleep} and pass data")
#                     time.sleep(error_sleep)
#                     pass
#                     # sys.exit()                   
                    
#                 try:
#                     s_json = jsonResponse.decode()
#                 except:
#                     logger.info(f"{getJsonResponse.__name__}(): Invalid response from Twitter")
#                     print("Invalid response from Twitter")
#                     pass
#                     # sys.exit()                    
#                 else:
#                     try:
#                         dataJson = json.loads(s_json)
#                         # return dataJson
#                     except:
#                         logger.info(f"{getJsonResponse.__name__}(): Json Parsing 에러, url: {url}")
#                         print("Error parsing JSON: %s" % s_json)
#                         pass
#                         # sys.exit()
                
#                 if dataJson:
#                     return dataJson

#             logger.info(f"{func.__name__}() : FAILED")
        
#         return getJsonResponse
#     return decorator

# # @retry(total_try_cnt=3, )


#                 # return dataJson
            

In [ ]:
# @retry(total_try_cnt=3, error_sleep=20, request_delay=2, retryable_exceptions=(TimeoutError))
# def crawl_tweets(start_date, end_date):    
#     got.manager.TweetManager.getJsonResponse = getJsonResponse # (tweetCriteria, refreshCursor, cookieJar, proxy, useragent=None, debug=False) # 함수에 이 부분 넣어주고
    
#     print("========== 트윗 수집 시작: {0} ~ {1} ==========".format(start_date, end_date))
#     start_time = time.time()
#     tweet_criteria = got.manager.TweetCriteria().setQuerySearch('Elon Musk')\
#                                                 .setSince(start_date)\
#                                                 .setUntil(end_date)\
#                                                 .setLang('en')
#     tweets = got.manager.TweetManager.getTweets(tweet_criteria, debug=True)
    
#     elapsed_time = time.time()-start_time
    
#     print("수집 완료 : {}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
#     print("총 수집 트윗 개수 : {0}".format(len(test_tweet)))
    
#     return tweets